In [32]:
from SPARQLWrapper import SPARQLWrapper, JSON
from datetime import datetime
import pandas as pd
import numpy as np

connect to db

In [33]:
sparql = SPARQLWrapper('https://dati.camera.it/sparql')

Create dataframe of the deputati from a legislation

In [34]:
dep = pd.read_csv('data/18esima_Leg/dep_18.csv')[['persona', 'nomeGruppo', 'nome', 'cognome']]
a = dep['cognome']+ ' ' + dep['nome']
dep['Parlamentare'] = a
dep.drop(['nome', 'cognome'], axis = 1, inplace= True)

a = []
for i in range(dep.shape[0]):
    a.append(dep['persona'][i][39:])

dep['id'] = a
dep.drop('persona', inplace= True, axis = 1)

p = []
dat = []
for i in dep['nomeGruppo']:
    e = i.split()
    p.append(e[-2][1:-1])
    d = e[-1].replace(')', '')
    dat.append(datetime.strptime(d[-10:], '%d.%m.%Y'))

dep['Partito'] = p
dep['dataIngresso'] = dat
dep.drop('nomeGruppo', axis = 1, inplace= True)

final_df = pd.DataFrame(columns= dep.columns)

for i in np.unique(dep['id']):
    if len(dep[dep['id'] == i]) < 2:
        final_df = pd.concat([final_df, dep[dep['id'] == i]])
    else:
        duplicate = dep[dep['id'] == i]
        final_df = pd.concat([final_df, duplicate.head(1)])


In [35]:
sparql.setReturnFormat(JSON)

leg = 18

# deputati of a specific legislation
sparql.setQuery("""
select distinct ?votazione ?titolo ?numeroVotazione ?cognome ?nome ?espressione ?infoAssenza ?deputato where {
## prima votazione del 8 febbraio 2022
?votazione a ocd:votazione;
   dc:date '20220208';
   rdfs:label ?titolo;
   dc:description ?descrizione;
   dc:identifier ?numeroVotazione. FILTER(regex(?numeroVotazione,'001$'))


## voti espressi
?voto a ocd:voto;
   ocd:rif_votazione ?votazione;
   dc:type ?espressione;
   ocd:rif_deputato ?deputato.
OPTIONAL{?voto dc:description ?infoAssenza}


## info deputato
?deputato foaf:surname ?cognome; foaf:firstName ?nome  


} ORDER BY ?cognome
 
"""
)


In [36]:
try:
    ret = sparql.queryAndConvert()
except Exception as e:
    print(e)

In [37]:
res = ret["results"]["bindings"]

In [48]:
dep = []
vot = []
for i in res:
    dep.append((i.get('deputato')).get('value')[40:46])
    vot.append((i.get('espressione')).get('value'))

    
    

In [49]:
pd.DataFrame([dep,vot], index = ['Deputato', 'Voto1']).T

,Deputato,Voto1
0,d307538_18,Favorevole
1,d307559_18,Non ha votato
2,d307394_18,Favorevole
3,d307119_18,Favorevole
4,d307114_18,Favorevole
...,...,...
623,d307143_18,Favorevole
624,d307616_18,Non ha votato
625,d305598_18,Favorevole
626,d307578_18,Favorevole


In [50]:
np.unique(vot)

array(['Astensione', 'Contrario', 'Favorevole', 'Non ha votato'],
      dtype='<U13')